In [2]:
import itertools
import numpy as np
from functools import partial, reduce
from qiskit import BasicAer, QuantumRegister, execute
from qiskit.quantum_info import Pauli
from qiskit_aqua import Operator, get_aer_backend
from qiskit_aqua.components.initial_states import Custom
from scipy.optimize import minimize
np.set_printoptions(precision=7, suppress=True)

Current and near-term quantum computers suffer from imperfections, as we repeatedly pointed it out. This is why we cannot run long algorithms, that is, deep circuits on them. A new breed of algorithms started to appear since 2013 that focus on getting an advantage from imperfect quantum computers. The basic idea is extremely simple: run a short sequence of gates where some gates are parametrized. Then read out the result, make adjustments to the parameters on a classical computer, and repeat the calculation with the new parameters on the quantum hardware. This way we create an iterative loop between the quantum and the classical processing units, creating classical-quantum hybrid algorithms.

<img src="figures/hybrid_classical_quantum.png" alt="Hybrid classical-quantum paradigm" style="width: 600px;"/>

These algorithms are also called variational to reflect the variational approach to changing the parameters. One of the most important example of this approach is the quantum approximate optimization algorithm, which is the subject of this notebook.

# 1) Quantum approximate optimization algorithm (QAOA)

The quantum approximate optimization algorithm (QAOA) is shallow-circuit variational algorithm for gate-model quantum computers that was inspired by quantum annealing. We discretize the adiabatic pathway in some $p$ steps, where $p$ influences precision. Each discrete time step $i$ has two parameters, $\beta_i, \gamma_i$. **The classical variational algorithms does an optimization over these parameters (gradient descent for example...) based on the observed energy at the end of a run on the quantum hardware**.  ``The idea is to learn`` $\beta_i, \gamma_i$ ``in order to find the optimal shortcut through the adiabatic process in terms of a finite number of unitary operations (to be calculated on a quantum computer``.

More formally, we want to discretize the time-dependent $H(t)=(1-t)H_0 + tH_1$ under adiabatic conditions. We achieve this by ``trotterizing`` the unitary; "Trotterization" = approximating a continuous Hamiltonian by the product of a large number of small discrete rotations, on 1 or 2 qubits at a time). The converse direction, proved by Aharonov et al. in 2004, is much more interesting: any quantum circuit can be simulated in the adiabatic model with only polynomial slowdown, provided you’re willing to allow a final Hamiltonian whose ground state is a superposition over the entire history of the circuit-model computation.

For instance, for time step $t_0$, we can split this unitary as 

$$U(t_0) \approx U(H_0, \beta_0)U(H_1, \gamma_0)$$

where the idea is to start by applying the first Hamiltonian $H_0$ for some duration $\beta_0$ followed by applying the second Hamiltonian $H_1$ for some time duration $\gamma_0$. We can continue doing this for subsequent time steps, eventually splitting up the evolution to $p$ such chunks:

$$
U \approx U(H_0, \beta_0)U(H_1, \gamma_0)\ldots U(H_0, \beta_p)U(H_1, \gamma_p).
$$

Obviously, this approximation will become better as one increases the number of steps $p$ and this approximation is known as ``trotterization`` (name comes from the "Suzuki-Trotter expansion").  At the end of optimizing the parameters, this discretized evolution will approximate the adiabatic pathway:

<img src="figures/qaoa_process.png" alt="Quantum approximate optimization algorithm" style="width: 400px;"/>

<img src="figures/trotter.png" alt="Trotter" style="width: 1000px;"/>

**The Hamiltonian $H_0$ is often referred to as the driving or mixing Hamiltonian, and $H_1$ as the cost Hamiltonian**. The simplest mixing Hamiltonian is $H_0 = -\sum_i \sigma^X_i$, the same as the initial Hamiltonian in quantum annealing. By alternating between the two Hamiltonians, the mixing Hamiltonian drives the state towards and equal superposition, whereas the cost Hamiltonian tries to seek its own ground state.

## Example with 2 qubits

Now we can define our mixing Hamiltonian on 2 qubits. As in the notebook on classical and quantum many-body physics, we had to define, for instance, an `IZ` operator to express $\mathbb{I}\otimes\sigma_1^Z$, that is, the $\sigma_1^Z$ operator acting only on qubit 1. We can achieve the same effect the following way (this time using the Pauli-X operator):

In Qiskit, Pauli matrices can be instantiated using the class `Pauli`. This class takes two parameters, the first for $\sigma^Z$ and the second for $\sigma^X$. Each parameter is a binary vector of dimension `n_qubits`, such that the component $i$ is 1 if you want a Pauli matrix to apply on the $i^{th}$ qubit and 0 otherwise. For instance, $\sigma_1^Z \otimes \sigma_3^Z \otimes \sigma_1^X$ would be implemented using `Pauli([1, 0, 1],[1, 0, 0])`.

In order to build Hamiltonians and make them evolve (i.e. exponentiate them, as required in QAOA), we need to use the class `Operator` from Qiskit Aqua. This class constructs a Hamiltonian as a sum of products of Pauli matrices. It takes an array of size $n \times 2$ as parameter, such that each row corresponds to a term in the sum and each term has two components: a coefficient and a `Pauli` object. For instance, $3 \sigma^Z_1 + 2 \sigma^X_3$ would be written `Operator([[3, Pauli([1, 0, 0], [0, 0, 0])], [2, Pauli([0, 0, 0],[0, 0, 1])]])`. 

To simplify the code, let's build a function `pauli_x` that simply takes a qubit and a coefficient and returns the corresponding X-Pauli matrix as an `Operator` 

In [3]:
n_qubits = 2

eye = np.eye((n_qubits))

pauli_x = lambda qubit, coeff:  Operator([[coeff, Pauli(np.zeros(n_qubits), eye[qubit])]])

### Define the mixing Hamiltonian

The coefficient here means the strength of the transverse field at the given qubit. This operator will act trivially on all qubits, except the given one. Let's define the mixing Hamiltonian $H_0 = - \sum_i \sigma^X_i = - ( \sigma_0^X + \sigma_1^X) $ over two qubits:

<img src="figures/signQuestion.png" alt="Trotter" style="width: 900px;"/>

In [4]:
Hm = reduce(lambda x, y: x+y, [pauli_x(i, 1) for i in range(n_qubits)])
Hm.to_matrix()

### Define the cost Hamiltonian

As an example, we will minimize the Ising problem defined by the cost Hamiltonian $H_1 = -\sigma^Z_1 \otimes \sigma^Z_2$. First let's create the functions defining the operators using the Pauli-Z matrix:

In [5]:
pauli_z         = lambda qubit, coeff: Operator([[coeff, Pauli(eye[qubit], np.zeros(n_qubits))]])
product_pauli_z = lambda q1, q2, coeff: Operator([[coeff, Pauli(eye[q1], np.zeros(n_qubits)) * Pauli(eye[q2], np.zeros(n_qubits))]])

Then we define the cost Hamiltonian:

In [6]:
J = np.array([[0, 1], [0, 0]])

allPermutations = list(itertools.product(range(n_qubits), repeat=2))
print(allPermutations)

Hc = reduce(lambda x,y: x + y, [product_pauli_z(i, j, -J[i, j]) for i, j in allPermutations])
Hc.to_matrix()

[(0, 0), (0, 1), (1, 0), (1, 1)]


### Introduce trainable parameters

We set $p=2$ and initialize the $\beta_i$ and $\gamma_i$ parameters

In [7]:
n_iter = 10 # number of iterations of the optimization procedure
p = 2       # number of "trotters"

beta = np.random.uniform(0, np.pi*2, p)
gamma = np.random.uniform(0, np.pi*2, p)

beta_gamma = np.concatenate([beta, gamma])

print('initial beta values = ', beta_gamma[:p])
print('initial gamma values = ', beta_gamma[p:])

initial beta values =  [0.5802356 0.2683601]
initial gamma values =  [6.0277403 1.6766796]


### Prepare initial part of circuit (state preparation to superposition)

The initial state is a uniform superposition of all the states $\sim |q_1,...,q_n\rangle$.  To declare the initial state, we use the Qiskit Aqua class `Custom`. It takes two arguments: the number of qubits of the state we want to prepare, and the vector containing the amplitudes.

In [8]:
init_state_vect = [1 for i in range(2**n_qubits)]
init_state = Custom(n_qubits, state_vector=init_state_vect)

The initial circuit prepares the initial state

In [9]:
qr = QuantumRegister(n_qubits)
circuit_init = init_state.construct_circuit('circuit', qr)

### Create circuit for quantum state evolution

We define a function `evolve` that takes a Hamiltonian $H$ and an angle $t$ and returns a circuit component made of the unitary matrix $e^{j H t}$.  For that, we use the method [`evolve`](https://github.com/Qiskit/qiskit-aqua/blob/master/qiskit/aqua/operator.py) of the class `Operator`. The arguments are:
* **initial circuit**: if we want to build $e^{iHt} |\psi\rangle$ with $|\psi\rangle$ an initial state. Set to `None` if we just need $e^{iHt}$, as in our case (we will append the initial circuit built above only at the end, not between all the exponentials).
* **angle**: the parameter t in $e^{iHt}$
* **type** of the returned object: in our case, we want a 'circuit'
* **quantum registers**: as usual
* **expansion_mode**: method used to compute the evolution
* **expansion_order**: order of the approximation used for computing the evolution

In [10]:
def evolve(hamiltonian, angle, quantum_registers):
    return hamiltonian.evolve(None, angle, 'circuit', 1,
                              quantum_registers=quantum_registers,
                              expansion_mode='suzuki',
                              expansion_order=3)

To create the circuit, we need to compose the different unitary matrice given by `evolve`.  (This can be achieved by using the same trick as above to sum the circuits using the reduce function).

Since we are going to consider the $\beta$'s and $\gamma$'s as trainable parameters, we need to parametrize the ``create_circuit`` function accordingly:

In [11]:
def create_circuit(beta, gamma):
    circuit_evolv = reduce(lambda x,y: x+y, [evolve(Hc, beta[i], qr) + evolve(Hm, gamma[i], qr) for i in range(p)])
    circuit = circuit_init + circuit_evolv
    return circuit

### Parametrized "cost" function that evaluates the circuit on QPU

Next, we create a function `evaluate_circuit` that takes a single vector `gamma_beta` (the concatenation of `gamma` and `beta`) and returns $\langle H_c \rangle = \langle \psi | H_c | \psi \rangle$ where $\psi$ is defined by the circuit created with the function above.  We use the method `eval` of the class `Operator` in order to compute $\langle \psi | H_c | \psi\rangle$. It takes the circuit of $|\psi\rangle$ and a backend as argument (as well as the method to use to perform the evaluation, here `"matrix"`)

In [12]:
def evaluate_circuit(beta_gamma):
    circuit = create_circuit(beta_gamma[:p], beta_gamma[p:])
    return np.real(Hc.eval("matrix", circuit, get_aer_backend('statevector_simulator'))[0])

### Optimization of parameters on CPU

Finally, we optimize the angles using the **``classical optimization routine from scipy``**:

In [14]:
result = minimize(evaluate_circuit, beta_gamma, method='L-BFGS-B')
result

      fun: -0.999999999999269
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0000006,  0.0000002,  0.0000048,  0.0000045])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 40
      nit: 6
   status: 0
  success: True
        x: array([0.6994744, 0.2817787, 5.9600443, 1.4893345])

# Analysis of the results

We create a circuit using the optimal parameters found.

In [15]:
circuit = create_circuit(result['x'][:p], result['x'][p:])

We use the `statevector_simulator` backend in order to display the state created by the circuit.

In [16]:
backend = BasicAer.get_backend('statevector_simulator')
job = execute(circuit, backend)
state = np.asarray(job.result().get_statevector(circuit))

np.absolute(state)

array([0.7071068, 0.0000004, 0.0000004, 0.7071068])

As hoped for, **``we see that the state is approximately``** $\frac{1}{\sqrt{2}} \left( |00 \rangle + |11 \rangle \right)$. 

It corresponds to a uniform superposition of the two solutions of the classical problem: $(\sigma_1=1$, $\sigma_2=1)$ and $(\sigma_1=-1$, $\sigma_2=-1)$.

Let's now try to evaluate the operators $\sigma^Z_1$ and $\sigma^Z_2$ independently:

In [17]:
Z0, Z1 = pauli_z(0, 1), pauli_z(1, 1)

print(Z0.eval("matrix", circuit, get_aer_backend('statevector_simulator'))[0])
print(Z1.eval("matrix", circuit, get_aer_backend('statevector_simulator'))[0])

1.1102230246251565e-16
1.1102230246251565e-16


We see that both are approximatively equal to zero. It's expected given the state we found above and corresponds a typical quantum behavior where $\mathbb{E}[\sigma^Z_1 \sigma^Z_2] \neq \mathbb{E}[\sigma^Z_1] \mathbb{E}[\sigma^Z_2]$.  

In other words, expectation values of products of operators do not equal products of expectation values of operators if you take the expectation value in a state that is entangled.

# 2) Another example of variational circuits (closer to traditional ML tasks)

Using automatic differentiation of quantum programs using [`PennyLane`](https://github.com/XanaduAI/pennylane).  More details in this [`medium post`](https://medium.com/xanaduai/training-quantum-neural-networks-with-pennylane-pytorch-and-tensorflow-c669108118cc) by Xanadu.

<img src="figures/PennyLane.png" alt="Hybrid classical-quantum paradigm" style="width: 600px;"/>